In [1]:
!pip install --upgrade pixiedust
import pixiedust
pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:2.3.3")


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210730044031-0000
KERNEL_ID = 55482fe3-0777-4d65-bb95-af0cb2fb50fe
     |████████████████████████████████| 197 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 62 kB 790 kB/s  eta 0:00:01
     |████████████████████████████████| 10.3 MB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 63.1 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 77.8 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 74.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 60.9 MB/s eta 0:00:01
     |███████████

Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully
PackageManager is disabled because /home/spark/shared/user-libs/spark2/ is not on the Java Class Path. Please contact your administrator


In [2]:
def readDataFrameFromCloudant(database):

    cloudantdata = spark.read.format("org.apache.bahir.cloudant")\
        .option("cloudant.host",'76dee3c7-edd9-4074-b1d9-65e3d1883319-bluemix.cloudantnosqldb.appdomain.cloud')\
        .option("cloudant.username", 'apikey-v2-1dvsabj9j0tb7d8l2epk3sqm7ek95w453qqtpt3h8ggs')\
        .option("cloudant.password",'5118b0774c3741f7c3cae78cd8a17acc')\
        .load(database)

    return cloudantdata


In [3]:
df = readDataFrameFromCloudant('trainingbathroom')
df.createOrReplaceTempView('df')
df = df.dropna()
spark.sql('select class, count(class) from df group by class').show()

+-----+------------+
|class|count(class)|
+-----+------------+
|    0|         268|
|    1|         260|
+-----+------------+



In [ ]:
display(df)

_id,_rev,ax,ay,az,class,time
05fc6d8f6ba1994ec67777e31e9fe92c,1-7c7d18ecf92e1e325382fbe413e6434a,2.86,30.66,101.51,1,1627619178780
05fc6d8f6ba1994ec67777e31ea00a99,1-f9755ce1e3d1e728db65217b86bb86ca,2.69,29.59,102.27,1,1627619180790
05fc6d8f6ba1994ec67777e31ea0195b,1-fe1166fc027a2f2261c1f02df87ebd4c,31.67,41.41,93.65,1,1627619183810
05fc6d8f6ba1994ec67777e31ea04770,1-670ff41348e31e779a0da23981facc5c,-65.53,31.1,81.27,1,1627619193405
05fc6d8f6ba1994ec67777e31ea05615,1-da0ccb1094eedb6cfe3f2f095faf6d65,-67.36,29.2,78.93,1,1627619200401
1a7249d5a6b1cd669a5da51ee5a279aa,1-6adcf36688cc0315b9138790f81f0a9e,-11.21,65.62,77.37,1,1627619159779
1a7249d5a6b1cd669a5da51ee5a282a3,1-383b87219f447008e5233791b050d5c8,-10.4,68.43,77.66,1,1627619159779
1a7249d5a6b1cd669a5da51ee5a2d738,1-407003b604ab085f34a045a90f7cebeb,14.58,38.48,99.76,1,1627619177783
1a7249d5a6b1cd669a5da51ee5a2dc8d,1-aa2176811918be8bc7e1789ea593c646,8.2,34.18,102.29,1,1627619177784
3df9aef08486f586ed815f66d72b90f4,1-ad94b72420e05d8b0322fae78765e2b8,-11.08,67.41,80.4,1,1627619168781


In [5]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [6]:
vectorAssembler = VectorAssembler(inputCols=["ax","ay","az"], outputCol="features")
lr = LogisticRegression(maxIter=1000).setLabelCol("class")
pipeline = Pipeline(stages=[vectorAssembler, lr ])
model = pipeline.fit(df)
result = model.transform(df)


In [8]:
print('intercept = ', model.stages[1].intercept)
print('coefficients = ', model.stages[1].coefficients)


intercept =  2.281447381186692
coefficients =  [0.06433991736957428,0.007893953859852535,0.01406214243619157]


In [9]:
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("class")
binEval.evaluate(result)


0.75

In [10]:
new_df = readDataFrameFromCloudant('training2')
new_df.createOrReplaceTempView('new_df')
result = model.transform(new_df)
result.createOrReplaceTempView('result')
spark.sql("select ax, ay, az, class, prediction from result").show(568)


+-------+-------+-------+-----+----------+
|     ax|     ay|     az|class|prediction|
+-------+-------+-------+-----+----------+
| 115.75| -17.19|-380.03|    1|       1.0|
|  -0.71|  57.76|  321.7|    1|       1.0|
| -43.73| -189.4| 204.49|    1|       1.0|
|  96.66|  -87.4| 183.91|    1|       1.0|
|   33.4|-339.89|  44.48|    1|       1.0|
| 120.68|    7.3|   96.8|    1|       1.0|
|  -1.76|   2.71|  107.1|    0|       1.0|
|  -1.32|   3.05|  107.4|    0|       1.0|
|  -2.39|   3.12| 108.91|    0|       1.0|
|  -1.61|   2.61| 107.42|    0|       1.0|
|  -2.44|   2.78| 107.69|    0|       1.0|
|  -1.93|   2.27| 106.67|    0|       1.0|
|   -3.0|   2.17| 106.91|    0|       1.0|
|  -2.29|   3.08| 107.54|    0|       1.0|
|  -2.59|   3.12| 106.69|    0|       1.0|
|  -1.59|   2.91| 107.57|    0|       1.0|
|  -2.54|   2.78| 106.79|    0|       1.0|
|-142.99|  42.24| 401.07|    1|       0.0|
| -77.05|-168.95| 351.51|    1|       1.0|
| -14.92| -38.45|  97.51|    1|       1.0|
|  54.71|  